# Introduction

In this notebook, I implement a neural network from scratch using PyTorch.I will implement it while training a digit classifier that can recognizes 10 handwritten digits. The dataset that I am using is the popular MNIST dataset.

Now, let's get started by loading the dataset from Fast AI's library.

In [1]:
!pip install fastai -q

In [2]:
from fastai.vision.all import *

Fast AI has a handy helper function to download and extract from the URL.

In [3]:
path = untar_data(URLs.MNIST)

In [4]:
Path.BASE_PATH = path

Listing the content inside the path object, there are two directories where the images are grouped into training and test dataset.

In [5]:
path.ls()

(#2) [Path('training'),Path('testing')]

Exploring what's inside the training directory, we see that each digit has its own directory. So the labels that we want to predict is indicated by the name of this sub-directory.



In [6]:
paths = (path / 'training').ls().sorted()
paths

(#10) [Path('training/0'),Path('training/1'),Path('training/2'),Path('training/3'),Path('training/4'),Path('training/5'),Path('training/6'),Path('training/7'),Path('training/8'),Path('training/9')]

Before loading the entire dataset, let's see a couple of the images.

In [7]:
zeros  = (path / 'training' / '0').ls()
Image.open(zeros[-1])

In [8]:
fives = (path/'training'/'5').ls()
im = Image.open(fives[-1])
im

The types of these images are PIL object.

In [9]:
type(im)

PIL.PngImagePlugin.PngImageFile

Instead, we want to covert them into PyTorch's tensor using the `tensor` method.

Each grey-scale image is represented by a 28x28 matrix of pixel vaues where they range from 0-25 (black to white).

Below you see a small section of these pixel values.

In [10]:
t = tensor(im)
t.shape, t[5:10, 10:15]

(torch.Size([28, 28]),
 tensor([[  0,   0,   0,   0,  24],
         [  0,   0,   0,  16, 186],
         [  0,   0,  16, 189, 251],
         [  0,   0, 171, 251, 251],
         [  0,  60, 228, 251, 211]], dtype=torch.uint8))

Create a list containing the images as tensors and their labels from training dataset.

In [11]:
train_xs = []
train_ys = []
tot = 0
for i, p in enumerate(paths):
  digit_paths = p.ls()
  num_digits = len(digit_paths)
  tot += num_digits
  train_ys += tensor([i] * num_digits)
  train_xs.extend([tensor(Image.open(f)) for f in digit_paths])

Let's sense check the total number of images, and the size of an individual elements in our xs and ys.

There are a total of 60,000 images and each image is a 28x28 pixel image and the label of the first image digit is zero.

In [13]:
len(train_xs), len(train_ys), train_xs[0].shape, train_ys[0]

(60000, 60000, torch.Size([28, 28]), tensor(0))

Now we can load the test data with the same code.

The size of the test dataset is 10,000.

In [14]:
test_paths = (path/'testing').ls().sorted()
test_xs = []
test_ys = []
tot_test = 0
for i, p in enumerate(test_paths):
  digit_paths = p.ls()
  num_digits = len(digit_paths)
  tot_test += num_digits
  test_ys += tensor([i] * num_digits)
  test_xs.extend([tensor(Image.open(f)) for f in digit_paths])

In [15]:
len(test_xs), len(test_ys), test_xs[0].shape, test_ys[0]

(10000, 10000, torch.Size([28, 28]), tensor(0))

# Preparing the data

Now that the data is loaded is a list of PyTorch, what we want to do next is to represent the entire dataset as PyTorch. For example, the training dataset is a tesnor of shape (60_000, 28, 28) and the training label is a tensor of shape (60_000, 1).

We can stack each of the digit on a row using the `stack` method, which works as follow:

In [16]:
import torch

In [17]:
a, b = tensor([2, 2]), tensor([9, 10])
torch.stack([a, b], dim=0)

tensor([[ 2,  2],
        [ 9, 10]])

Let's convert our training and test dataset from list to tensors.

In [18]:
xs_train, ys_train = torch.stack(train_xs, dim=0), torch.stack(train_ys)
xs_test, ys_test = torch.stack(test_xs), torch.stack(test_ys)

Checking the shape.

In [19]:
xs_train.shape, ys_train.shape, xs_test.shape, ys_test.shape

(torch.Size([60000, 28, 28]),
 torch.Size([60000]),
 torch.Size([10000, 28, 28]),
 torch.Size([10000]))

If you recall from earlier, the pixel values are integer and cover a wide range of numbers.

In [20]:
xs_train[0, 5:15, 5:15]

tensor([[  0,   0,   0,   0,   0,  25, 231, 252, 252, 252],
        [  0,   0,   0,   0,  14, 131, 252, 252, 252, 252],
        [  0,   0,   0,   0, 223, 252, 252, 252, 252, 252],
        [  0,   0,   5, 145, 248, 252, 252, 252, 252, 207],
        [  0,   0, 121, 252, 252, 252, 252, 241, 195,  30],
        [  0,  32, 210, 252, 252, 252, 252,  84,   0,   0],
        [  0, 113, 252, 252, 252, 241,  84,   1,   0,   0],
        [ 19, 216, 252, 252, 252, 108,   0,   0,   0,   0],
        [105, 252, 252, 252, 207,  30,   0,   0,   0,   0],
        [106, 253, 253, 253, 117,   0,   0,   0,   0,   0]], dtype=torch.uint8)

However, the issue is that neural networks work better with smaller decimal numbers.

Since we know that 255 is the maximum pixel values, we will scale each of these pixel values to be in the range of 0-1 by dividing each of the values by its maximum (A process known as **normalization**).

In [21]:
xs_train = xs_train.float() / 255
xs_test = xs_test.float() / 255

In [22]:
xs_train[0, 5:15, 5:15]

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0980, 0.9059, 0.9882, 0.9882,
         0.9882],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0549, 0.5137, 0.9882, 0.9882, 0.9882,
         0.9882],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.8745, 0.9882, 0.9882, 0.9882, 0.9882,
         0.9882],
        [0.0000, 0.0000, 0.0196, 0.5686, 0.9725, 0.9882, 0.9882, 0.9882, 0.9882,
         0.8118],
        [0.0000, 0.0000, 0.4745, 0.9882, 0.9882, 0.9882, 0.9882, 0.9451, 0.7647,
         0.1176],
        [0.0000, 0.1255, 0.8235, 0.9882, 0.9882, 0.9882, 0.9882, 0.3294, 0.0000,
         0.0000],
        [0.0000, 0.4431, 0.9882, 0.9882, 0.9882, 0.9451, 0.3294, 0.0039, 0.0000,
         0.0000],
        [0.0745, 0.8471, 0.9882, 0.9882, 0.9882, 0.4235, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.4118, 0.9882, 0.9882, 0.9882, 0.8118, 0.1176, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.4157, 0.9922, 0.9922, 0.9922, 0.4588, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000]])

Currently, our labels takes on the values between 0 and 9.

Once again, the neural network can't simply predict an integer label for us. Whenever we have multiple labels, one approach we can take is to represent them as one-hot encoding.Here's how it works.

**One-hot encoding**

Suppose I have three labels (car, truck, motorcycles). One hot encoding represents each of the label with a list of numbers (or vectors) as follows:

1. Assign a number/index to each label. For example - Car:0, Truck:1, Motorcycles:2
2. Then each label is represented as vector with the size of maximum number of possible labels (i.e. 3 in this case).
3. To represent each label, I store 1 in the appropriate location of the vector that this label is assigned to and zero in all other positions. So this is a vector representing a car `[1, 0, 0]`

Subsequently, truck and motorcyles are `[0, 1, 0]` and `[0, 0, 1]`.

In [23]:
ys_train

tensor([0, 0, 0,  ..., 9, 9, 9])

There is a function in PyTorch called `one_hot` that does it for us.

Below you will see it looks like for the first and last few rows (digit 0 and 9)

In [24]:
import torch.nn.functional as F

In [25]:
ys_train_enc = F.one_hot(ys_train, num_classes=10)
ys_train_enc.shape, ys_train_enc

(torch.Size([60000, 10]),
 tensor([[1, 0, 0,  ..., 0, 0, 0],
         [1, 0, 0,  ..., 0, 0, 0],
         [1, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 1],
         [0, 0, 0,  ..., 0, 0, 1],
         [0, 0, 0,  ..., 0, 0, 1]]))

In [26]:
ys_test_enc = F.one_hot(ys_test, num_classes=10)
ys_test_enc.shape, ys_test_enc

(torch.Size([10000, 10]),
 tensor([[1, 0, 0,  ..., 0, 0, 0],
         [1, 0, 0,  ..., 0, 0, 0],
         [1, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 1],
         [0, 0, 0,  ..., 0, 0, 1],
         [0, 0, 0,  ..., 0, 0, 1]]))

Finally, when passing our training images to the network, we can't pass 28x28 pixels as a matrix instead we have to flatten the matrix into a vector. We can do that using `view` method on the tensor

In [27]:
input_size = 28*28
xs_train = xs_train.view(-1, input_size)
xs_test = xs_test.view(-1, input_size)

In [28]:
xs_train.shape, xs_test.shape

(torch.Size([60000, 784]), torch.Size([10000, 784]))

In order to pass images in batches, we will make use of the `DataLoader` class from PyTorch, which expects a dataset in the form of tuples (x, y). We can use zip function to get it.

In [29]:
train_dataset, test_dataset = list(zip(xs_train, ys_train_enc)), list(zip(xs_test, ys_test_enc))

Each sample going into the neural network is a tensor with 784 values and the labels are now 10-dimensional vector.

In [30]:
x, y = train_dataset[0]
x.shape, y

(torch.Size([784]), tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

Defining the DataLoader to draw one batch of input images at a time. This will allow us to pass 120 images to the neural network in one training iteration.

In [31]:
train_dl = DataLoader(train_dataset, batch_size=120)
test_dl = DataLoader(test_dataset, batch_size=120)

We can peek a the first batch.

In [32]:
xs_first, ys_first = first(train_dl)
xs_first.shape, ys_first.shape

(torch.Size([120, 784]), torch.Size([120, 10]))

# Loss Function Formulation

Before we can build a neural network, one thing we need is a way to provide feedback to our network.

In other words, when we show an image, such as digit 0, to our network but our network predicts something else, we will need to provide a score that indicates whether it's doing good or bad. A function that does it is called a **loss function**.

How can we construct such a function? Remember how we represented our labels as one-hot encoded vector.

Going back to earlier [car, truck, and motorcycle] example, suppose we pass an image of car to our network but the neural network predicts a truck (each of the values in neural network predictions `nn_preds` can be interpreted as probability that the neural network thinks it belongs to a specific class):


In [33]:
car_label = tensor([1, 0, 0])
nn_preds = tensor([0.2, 0.7, 0.1])

When we have multi-classification problem, we can compute the loss by taking the negative log of the predictions across all the classes and summing them up.

This loss value should go down when the neural network gets better at predicting correctly.

In [37]:
-(car_label * torch.log(nn_preds)).sum()

tensor(1.6094)

Suppose our neural networks predicts the car with more confidence even though it's still getting it wrong (Probability of a car has increased from 0.2 to 0.4). Consequently, the loss went down from 1.6 to 0.9 indicating that this prediction is better!

In [38]:
nn_preds = tensor([0.4, 0.5, 0.1])
-(car_label * torch.log(nn_preds)).sum()

tensor(0.9163)

If it gets it right, the loss should go down even further.

In [39]:
nn_preds = tensor([0.7, 0.1, 0.2])
-(car_label * torch.log(nn_preds)).sum()

tensor(0.3567)

Quite often, instead of sending in one image at a time, we often send a batch of images to neural network.

In that case, we can take the loss to be an average of loss across the items in the batch.

In [40]:
car_labels = tensor([
    [1, 0, 0],
    [1, 0, 0],
    [1, 0, 0]
])
nn_predictions = tensor(
    [
     [0.2, 0.7, 0.1],
     [0.4, 0.5, 0.1],
     [0.7, 0.1, 0.2]
    ]
)
losses = -(car_labels * torch.log(nn_predictions)).sum(dim=1)
avg_loss = losses.sum() / car_labels.shape[0]
print(f"Losses: {losses}, Average Loss: {avg_loss}")

Losses: tensor([1.6094, 0.9163, 0.3567]), Average Loss: 0.9608011841773987


Now we can define the cross-entropy loss in a function.

In [41]:
def cross_entropy_loss(true_labels, predictions, printLoss=False):
  losses = -(true_labels * torch.log(predictions+1e-15)).sum(dim=1)
  average_loss = losses.sum() / true_labels.shape[0]
  if printLoss: print(f"Loss: {average_loss.item():.4f}")
  return average_loss

In [42]:
cross_entropy_loss(car_labels, nn_predictions, True)

Loss: 0.9608


tensor(0.9608)

# Constructing the Neural Network

To begin with, I will create a neural network with one layer.

## Initialize the parameters randomly

In [44]:
def init_params(size, std=1.0):
  return (torch.randn(size)*std).requires_grad_()

Now we have our weights & biases.

In [46]:
weights = init_params((input_size, 10))
biases = init_params(10)

In [47]:
weights.shape, biases.shape

(torch.Size([784, 10]), torch.Size([10]))

The shape of each incoming batch will have the following shape: `[batch_size, input_size]`.

In [48]:
xs_first.shape

torch.Size([120, 784])

## Get the predictions

We can get the predictions by performing matrix multiplication of the input & the weights and adding biases to it.

In [50]:
def linear(xb): return xb@weights + biases

In [52]:
preds = linear(xs_first)
preds.shape, preds[:1]

(torch.Size([120, 10]),
 tensor([[  7.6349,   4.5616,  24.0088,  16.5575,  -3.5533,   0.5458,  13.9238,
           -2.8771, -14.8012,  -2.2896]], grad_fn=<SliceBackward0>))

The predictions that we get aren't quite the probabilities like we expected and we need probabilities a vector of probabilities to pass into our loss function.

Let's see how we can achieve it. First let's take the exponent of our vector.

In [53]:
preds_exp = torch.exp(preds)
preds_exp.shape, preds_exp[:1]

(torch.Size([120, 10]),
 tensor([[2.0691e+03, 9.5736e+01, 2.6723e+10, 1.5518e+07, 2.8631e-02, 1.7259e+00,
          1.1144e+06, 5.6297e-02, 3.7317e-07, 1.0130e-01]],
        grad_fn=<SliceBackward0>))

Second, we sum up all the elements in the vector after taking the exponent.

In [ ]:
total_ = preds_exp.sum(dim=1)
total_.shape, total_[0]

(torch.Size([120]), tensor(1054781.5000, grad_fn=<SelectBackward0>))

Third, we divide the exponent of our vector by the sum we just computed.

However, the issue is the sum has one fewer dimension and it won't be able to broadcast if we want to divide. Therefore, we will specify `keepdims=True` when performing the sum. Let's see the resulting shape.

In [54]:
total_ = preds_exp.sum(dim=1, keepdims=True)
total_.shape, total_[0]

(torch.Size([120, 1]), tensor([2.6740e+10], grad_fn=<SelectBackward0>))

Great! Now, we can perform element-wise division correctly (via broadcasting).

In [55]:
probs = preds_exp / preds_exp.sum(dim=1, keepdims=True)
probs.shape, probs[0, :], probs[0, :].sum()

(torch.Size([120, 10]),
 tensor([7.7378e-08, 3.5803e-09, 9.9938e-01, 5.8035e-04, 1.0707e-12, 6.4544e-11,
         4.1674e-05, 2.1053e-12, 1.3956e-17, 3.7885e-12],
        grad_fn=<SliceBackward0>),
 tensor(1.0000, grad_fn=<SumBackward0>))

Since there are 120 images in each batch, the sum of the probabilities is 120. So, we know that we have correctly computed the probabilities.

In [56]:
probs.sum()

tensor(120., grad_fn=<SumBackward0>)

Taking the exponent of the numerical output of the neural network and dividing it by its sum is called the **softmax** function. Let's put this in a function.

In [57]:
def softmax(x):
  exponential = torch.exp(x)
  summation_classes = exponential.sum(dim=1, keepdims=True)
  return exponential / summation_classes

Now that our neural network is predicting probabilities, we can find the loss.

In [58]:
loss = cross_entropy_loss(true_labels=ys_first, predictions=probs, printLoss=True)

Loss: 7.5432


## Compute the gradients and perform back-propogation.

By calling `backward` on our loss functions, PyTorch will compute the gradients for all the parameters.

In [59]:
loss.backward()

These gradients will be accessible in `grad` attribute of the parameters.

In [60]:
weights.grad.shape, weights.grad.mean(), biases.grad

(torch.Size([784, 10]),
 tensor(6.0821e-10),
 tensor([-9.4333e-01,  1.2496e-01,  3.8489e-01,  2.0558e-01,  1.5233e-02,
          9.1414e-03,  1.6477e-01,  7.5494e-03,  2.7125e-05,  3.1173e-02]))

Now we can put this step into a function.

In [61]:
def find_gradient(xb, yb, model):
  predictions = model(xb)
  probabilities = softmax(predictions)
  loss = cross_entropy_loss(yb, probabilities)
  loss.backward()

If we call this fucntion a couple of times, you will notice that the gradient starts accumulating.

In [62]:
find_gradient(xs_first, ys_first, linear)
weights.grad.mean(), biases.grad

(tensor(1.2164e-09),
 tensor([-1.8867e+00,  2.4993e-01,  7.6977e-01,  4.1116e-01,  3.0465e-02,
          1.8283e-02,  3.2954e-01,  1.5099e-02,  5.4249e-05,  6.2346e-02]))

In [63]:
find_gradient(xs_first, ys_first, linear)
weights.grad.mean(), biases.grad

(tensor(1.4597e-09),
 tensor([-2.8300e+00,  3.7489e-01,  1.1547e+00,  6.1674e-01,  4.5698e-02,
          2.7424e-02,  4.9432e-01,  2.2648e-02,  8.1374e-05,  9.3519e-02]))

However, that's not the desirable behavior in neural network training. Once we have adjusted the weights in one training iteration, we want to reset it zero.

We can achieve this by calling `.zero` on our grad attribute.

In [64]:
weights.grad.zero_()
biases.grad.zero_();

We can put all the training steps of one iteration into a function.

In [65]:
def train_epoch(model, learning_rate, parameters):
  for xb, yb in train_dl:
    find_gradient(xb, yb, model)
    for p in parameters:
      p.data -= p.grad * learning_rate
      p.grad.zero_()

During the trianing, besides the loss, we also care about other metrics.

For example, we might want to know the accuracy of our classifier.

Let's try to write the code to find the accuracy with sample labels and predictions as follow:

1. Get the index of the maximum value in our predictions and true labels.
2. If these two indexes are same, then it's correct. Otherwise, it's wrong.

Since we are dealing with multiple items (or images) at a time, we can get the accuracy for the entire batch by taking the mean.

In [67]:
car_labels = tensor([
    [1, 0, 0],
    [1, 0, 0],
    [1, 0, 0]
])
nn_predictions = tensor(
    [
     [0.2, 0.7, 0.1],
     [0.4, 0.5, 0.1],
     [0.7, 0.1, 0.2]
    ]
)

_, predicted_labels = torch.max(nn_predictions, 1)
print(f"Predicted: {predicted_labels}")
_, true_car_labels = torch.max(car_labels, 1)
print(f"True: {true_car_labels}")
(predicted_labels == true_car_labels).float().mean()

Predicted: tensor([1, 1, 0])
True: tensor([0, 0, 0])


tensor(0.3333)

Now we can put it into a function to compute the accuracy for each batch.

In [68]:
def batch_accuracy(predictions_batch, labels_batch):
  _, predicted_labels = torch.max(predictions_batch, 1)
  _, true_labels = torch.max(labels_batch, 1)
  return (predicted_labels == true_labels).float().mean()

Testing on dummy dataset.

In [69]:
batch_accuracy(nn_predictions, car_labels)

tensor(0.3333)

In each iteration of the training, we would like to compute the accuracy on the test set.

In [70]:
def validate_epoch(model):
  accs = []
  for xb, yb in test_dl:
    accs.append(batch_accuracy(model(xb), yb))
  return round(torch.stack(accs).mean().item() , 4)

In [71]:
validate_epoch(linear)

0.1443

## Train until a stopping criterion is reached.

We can now train the whole thing from the scratch until a criterion we specify is reached.For now, I can train for 25 iterations.

In [74]:
lr = 1
weights = init_params((input_size, 10))
biases = init_params(10)
params = weights, biases
for i in range(25):
  train_epoch(linear, 0.1, params)
  print(validate_epoch(linear), end=" ")

0.1399 0.23 0.3362 0.4051 0.4504 0.4823 0.506 0.523 0.5386 0.5509 0.5593 0.566 0.5752 0.5814 0.5874 0.5928 0.5968 0.6013 0.6053 0.6076 0.6104 0.6126 0.6141 0.6164 0.6192 

We achieved an validation accuracy of 61%. There is one serious limiting factor on our network.

Currently, our neural network is only able to learn linear function. So how do we fix that? We will fix it by adding a non-linear function to the output of the linear model (in particular a function called ReLU).

## Non-Linearity

PyTorch has built-in ReLU funtion.

All it does is return zero when the input is negative. Otherwise, it returns the input itself.

In [76]:
import torch.nn.functional as F
F.relu(tensor([-3, 1, 4]))

tensor([0, 1, 4])

Once we did the matrix multiplication, all we need to do it pass it through the ReLU function.

We pass the linear output of the first batch into ReLU.

In [79]:
F.relu(linear(xs_first))

tensor([[32.4036,  0.0000, 12.6207,  ...,  0.0000, 21.2366,  3.4717],
        [25.3043,  0.0000,  1.8662,  ..., 10.2617,  8.3608,  7.2364],
        [19.2242,  0.0000,  2.9971,  ...,  0.0000, 12.4211, 16.1132],
        ...,
        [18.5834,  0.0000,  2.5219,  ...,  0.0000, 10.2877,  7.6897],
        [26.3569,  0.0000, 13.6052,  ...,  0.0000, 18.3796,  0.0000],
        [ 7.7505,  0.0000,  1.3770,  ...,  7.1471,  4.7673,  0.0000]],
       grad_fn=<ReluBackward0>)

Let's define our neural network again to include the ReLU as well.

In [80]:
def neural_net(xb):
  res = linear(xb)
  return F.relu(res)

Now we will try the training again for 25 iterations with this.

In [81]:
lr = 1
weights = init_params((input_size, 10))
biases = init_params(10)
params = weights, biases
for i in range(25):
  train_epoch(neural_net, 0.1, params)
  print(validate_epoch(neural_net), end=" ")

0.2364 0.2571 0.3437 0.4529 0.524 0.5639 0.5917 0.6135 0.6297 0.6412 0.6506 0.659 0.6664 0.6726 0.6792 0.683 0.686 0.6906 0.6953 0.6979 0.7008 0.7041 0.7075 0.7092 0.7106 

An accuracy of 71%! Clearly, our neural network is more powerful now compared to the linear model before and maybe if we train for longer we can even achieve better performance.

# Deep Neural Network

Now that we have written all the components to train one layer neural network, let's try to put everything together again and extend our code for handling multiple layers so we can build deep neural network.

While we had one matrix to store the parameters of a layer, we will now need to have mutiple matrices to represent multiple layers.

I will rewrite the function to initialize parameters from earlier. This function will:

1. Take a list of number of units that the user wants at each layer (The input layer should have 784 units and the final layer should have 10 units since there are 10 classes while the hidden units can be arbitary).
2. Initialize the weights & biases as before for each layers (I have used some heuristic for weight initializtion that works better - all it does it scale the weights a little better).
3. Tells PyTorch that these parameters will be required for gradient computation later on.

In [199]:
def init_params(sizes: list):
  n = len(sizes) # Number of layers as length of the sizes
  weights = [(torch.rand(sizes[i], sizes[i+1])-0.3) / sizes[i+1]*4 for i in range(n-1)]
  biases = [torch.rand(sizes[i+1])*0.1 for i in range(n-1)]
  for params in weights+biases: params.requires_grad_()
  return weights, biases

Let's double-check its correct.

In [138]:
w, b = init_params([4, 2, 2, 1])
for a, b in zip(w, b):
  print(a.shape, b.shape)

torch.Size([4, 2]) torch.Size([2])
torch.Size([2, 2]) torch.Size([2])
torch.Size([2, 1]) torch.Size([1])


Once we have parameters initialized, we will write a function to do the forward pass (i.e. send the images through the whole network and get the prediction.) through the model.

Similar to earlier, we will receive input of shape `[batch_size, input_size]`.

In [169]:
def neural_network(input_batch, params):
  weights, biases = params
  n = len(weights) # Find out the number of layers
  output = input_batch
  for i, (layer_weights, layer_biases) in enumerate(zip(weights, biases)):
    output = output @ layer_weights + layer_biases
    if i != n-1: output = F.relu(output)
  return softmax(output)

Next, I will write one more function that computes the gradients and perform back-propogation by updating the gradients.

In [159]:
def backprop(true_labels, predictions, lr, params):
  # Compute the loss
  loss = cross_entropy_loss(true_labels, predictions)
  loss.backward()
  with torch.no_grad():
    weights, biases = params
    for layer_params in weights+biases:
      layer_params.sub_(layer_params.grad * lr)
      layer_params.grad.zero_()

Now we write a function to train one epoch as before.

In [160]:
def train_epoch(model, learning_rate, params):
  for xs, ys in train_dl:
    predictions = model(xs, params)
    backprop(ys, predictions, learning_rate, params)

Modify the validate epoch function from earlier to take `params` as input.

In [161]:
def validate_epoch(model, params):
  accs = []
  for xb, yb in test_dl:
    accs.append(batch_accuracy(model(xb, params), yb))
  return round(torch.stack(accs).mean().item() , 4)

Let's start training our deep neural network with hidden layers.

In [207]:
lr = 0.1
epochs = 200
parameters = init_params([input_size, 100, 10])
for i in range(epochs):
  train_epoch(neural_network, lr, parameters)
  print(f" Iteration {i} , Validation Accuracy: {validate_epoch(neural_network, parameters)}")

 Iteration 0 , Validation Accuracy: 0.108
 Iteration 1 , Validation Accuracy: 0.1198
 Iteration 2 , Validation Accuracy: 0.1453
 Iteration 3 , Validation Accuracy: 0.1829
 Iteration 4 , Validation Accuracy: 0.233
 Iteration 5 , Validation Accuracy: 0.2729
 Iteration 6 , Validation Accuracy: 0.2919
 Iteration 7 , Validation Accuracy: 0.2988
 Iteration 8 , Validation Accuracy: 0.2895
 Iteration 9 , Validation Accuracy: 0.3142
 Iteration 10 , Validation Accuracy: 0.327
 Iteration 11 , Validation Accuracy: 0.3123
 Iteration 12 , Validation Accuracy: 0.3064
 Iteration 13 , Validation Accuracy: 0.3113
 Iteration 14 , Validation Accuracy: 0.315
 Iteration 15 , Validation Accuracy: 0.3487
 Iteration 16 , Validation Accuracy: 0.393
 Iteration 17 , Validation Accuracy: 0.3971
 Iteration 18 , Validation Accuracy: 0.3817
 Iteration 19 , Validation Accuracy: 0.3637
 Iteration 20 , Validation Accuracy: 0.3815
 Iteration 21 , Validation Accuracy: 0.4098
 Iteration 22 , Validation Accuracy: 0.4296
 It

# Final Thought

While I manage to get around 94% validation accuracy after 200 rounds of training (it still seems to be increasing with more training iteration), I find that it's quite difficult to please neural network and get that correct configuration to work.